In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from fpipe.pipeline.pipeline import run_pipeline
from fpipe.timestream import data_base as db
from fpipe.timestream import bandpass_cal
from fpipe.plot import plot_bandpass, plot_waterfall
import numpy as np
import os

data_base = '/home/ycli/data/fanalysis/'
#band_list = ['_1050-1150MHz', '_1150-1250MHz', '_1250-1450MHz']
band_list = ['_1050-1150MHz', '_1150-1250MHz',]
tnoise_file = '/home/ycli/data/fast/tnoise/Tnoise_M_low.h5'


In [3]:
@run_pipeline
def output_bandpass():
    pipe_tasks = []
    pipe_outdir = data_base
    pipe_logging = 'info'
    pipe_copy = False
    pipe_feedback = 0
    pipe_timing = True
    
    feed_select = list(np.arange(1, 20))
    #bad_feed = [14, 16, 19]
    #for x in bad_feed:
    #    feed_select.remove(x)
    
    pipe_tasks.append(bandpass_cal.OutputBandpass)
    
    #bpcal_input_files  = ['raw/%s'%input_file, ]
    #bpcal_output_files = ['bandpass/%s'%input_file, ]
    bpcal_noise_on_time = 1
    bpcal_bandpass_smooth = 101
    bpcal_pol_select = (0, 2)
    #bpcal_freq_select = (5000, -1000)
    #bpcal_freq_select = (5000, 7048)
    bpcal_feed_select = feed_select #[1,]
    bpcal_main_data = 'vis'
    bpcal_corr = 'auto'
    bpcal_bad_freq_list = None #bad_freq_list # [[0, 50], [595, 605], [930, 955]] #bad_freq
    bpcal_bad_time_list = None #bad_time_list # [[0, 120], [3150, 3180], [3750, 3820]]
    bpcal_Tnoise_file = tnoise_file
    
    return locals()

@run_pipeline
def cal():
    pipe_tasks = []
    pipe_outdir = data_base
    pipe_logging = 'info'
    pipe_copy = False
    pipe_feedback = 0
    pipe_timing = True
    
    feed_select = list(np.arange(1, 20))
    
    pipe_tasks.append(bandpass_cal.Bandpass_Cal)
    
    bpcal_input_files  = [] #['raw/%s'%(f, ) for f in input_files]
    bpcal_output_files = [] #['bpcal/%s'%f for f in input_files]
    bpcal_noise_on_time = 1
    bpcal_bandpass_file = '' #data_base + '/bandpass/%s/bandpass_%s.h5'%(DATA_Key, DATA)
    bpcal_pol_select = (0, 2)
    #bpcal_freq_select = (5000, -1000)
    #bpcal_freq_select = (5000, 7048)
    bpcal_feed_select = feed_select #[1,]
    bpcal_main_data = 'vis'
    bpcal_corr = 'auto'
    #bpcal_chunk_vis = True
    #bpcal_chunk_size = 100 * 2**10
    bpcal_bad_freq_list = None #bad_freq_list # [[0, 50], [595, 605], [930, 955]] #bad_freq
    bpcal_bad_time_list = None #bad_time_list # [[0, 120], [3150, 3180], [3750, 3820]]
    
    return locals()

@run_pipeline
def plot_wf():
    pipe_tasks = []
    pipe_outdir = data_base
    pipe_logging = 'info'
    pipe_copy = False
    pipe_feedback = 0
    pipe_timing = True
    
    feed_select = list(np.arange(1, 20))

    pipe_tasks.append((plot_waterfall.PlotMeerKAT, 'pkat1_'))
    
    pkat1_input_files  = [] #['%s/%s'%('bpcal', f,) for f in input_files]
    pkat1_re_scale  = 2.0 #2.0 #0.2
    pkat1_main_data = 'vis'
    pkat1_fig_name  = '' #'wf/%s_%s%s'%('bpcal', DATA_Key, suffix)
    pkat1_flag_mask = True
    pkat1_flag_ns   = True
    pkat1_flag_raw  = False
    pkat1_plot_ra = False
    pkat1_bad_freq_list = None #[[2600, 2800],]
    pkat1_bad_time_list = None #bad_time_list # [[0, 120], [3150, 3180], [3750, 3820]]
    pkat1_feed_select = [1, ] #feed_select #[show + 1, ]
    pkat1_plot_index  = False
    pkat1_unit = r'$T[K]$'
    
    return locals()

In [4]:
DATA_Key_list = [#'20210313', '20210314', '20210305', '20210302', '20210306', '20210307', '20210309', 
                 #'20220210', '20220211', '20220212', 
                 '20220213', '20220214', '20220222',
                 '20220215', '20220216', '20220217', '20220218', '20220219', '20220220',
                ]
def iter_data_key(DATA_Key_list):
    
    for DATA_Key in DATA_Key_list:
    
        data = db.DATA_BASE[DATA_Key]
        DATA = data['DATA']
        DATE = data['DATE']
        
        yield DATA_Key, DATA, DATE

In [5]:
for DATA_Key, DATA, DATE in iter_data_key(DATA_Key_list):

    file_temp = '%s/%s_arcdrift%04d-%04d%s.h5'
    
    for suffix in band_list:
        
        for ID in range(1, 8):
    
            input_file = file_temp%(DATE, DATA, ID, ID, suffix)
        
            output_bandpass['bpcal_input_files'] = ['raw/%s'%input_file, ]
            output_bandpass['bpcal_output_files']= ['bandpass/%s'%input_file, ]
            #output_bandpass()
            
            # flash memory
            %reset -f array

In [6]:
for DATA_Key, DATA, DATE in iter_data_key(DATA_Key_list):

    bandpass_path = data_base + 'bandpass/%s/'%DATE
    output_path = data_base + 'bandpass/%s/'%DATE
    plot_path = data_base + 'plot/bandpass_%s_%s_'%(DATA, DATE)
    
    bandpass_cal.output_smoothed_bandpass(bandpass_path, DATA, tnoise_file, 
                                          output_path = output_path, 
                                          blk_st=1, blk_ed=7)
    %reset -f array
    
    #plot_bandpass.plot_bandpass(bandpass_path, DATA, pol=0, ymin=0.97, ymax=1.029, 
    #                            normalize=True, ratio=True, output_path = plot_path)
    #%reset -f array
    #
    #plot_bandpass.plot_bandpass(bandpass_path, DATA, pol=1, ymin=0.97, ymax=1.029, 
    #                            normalize=True, ratio=True, output_path = plot_path)
    #%reset -f array
    #
    #plot_bandpass.plot_bandpass(bandpass_path, DATA, pol=0, ymin=0.71, ymax=1.29, 
    #                            normalize=True, ratio=False, output_path = plot_path)
    #%reset -f array
    #
    #plot_bandpass.plot_bandpass(bandpass_path, DATA, pol=1, ymin=0.71, ymax=1.29, 
    #                            normalize=True, ratio=False, output_path = plot_path)
    #%reset -f array

HIMGS_1100+2748_arcdrift0001-0001
HIMGS_1100+2748_arcdrift0002-0002
HIMGS_1100+2748_arcdrift0003-0003
HIMGS_1100+2748_arcdrift0004-0004
HIMGS_1100+2748_arcdrift0005-0005
HIMGS_1100+2748_arcdrift0006-0006
HIMGS_1100+2748_arcdrift0007-0007
HIMGS_1100+2759_arcdrift0001-0001
HIMGS_1100+2759_arcdrift0002-0002
HIMGS_1100+2759_arcdrift0003-0003
HIMGS_1100+2759_arcdrift0004-0004
HIMGS_1100+2759_arcdrift0005-0005
HIMGS_1100+2759_arcdrift0006-0006
HIMGS_1100+2759_arcdrift0007-0007
HIMGS_1100+2715_arcdrift0001-0001
HIMGS_1100+2715_arcdrift0002-0002
HIMGS_1100+2715_arcdrift0003-0003
HIMGS_1100+2715_arcdrift0004-0004
HIMGS_1100+2715_arcdrift0005-0005
HIMGS_1100+2715_arcdrift0006-0006
HIMGS_1100+2715_arcdrift0007-0007


In [ ]:
'''
Each band requires about 70GB memory, except band 1250-1450, need 120GB
'''
for DATA_Key, DATA, DATE in iter_data_key(DATA_Key_list):
    
    file_temp = '%s/%s_arcdrift%04d-%04d%s.h5'
    bandpass_path = data_base + 'bandpass/%s/'%DATE
    
    for suffix in band_list:
        input_files = [file_temp%(DATE, DATA, i, i, suffix) for i in range(1, 8)]
        cal['bpcal_input_files'] = ['raw/%s'%(f, ) for f in input_files]
        cal['bpcal_output_files'] = ['bpcal/%s'%f for f in input_files]
        cal['bpcal_bandpass_file'] = bandpass_path + 'bandpass_%s.h5'%DATA
        cal()
        %reset -f array

INFO:tlpipe.pipeline.pipeline:Initializing task: <class 'fpipe.timestream.bandpass_cal.Bandpass_Cal'>
INFO:tlpipe.pipeline.pipeline:Bandpass_Cal reading data from files:
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0001-0001_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0002-0002_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0003-0003_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0004-0004_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0005-0005_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0006-0006_1050-1150MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0007-0007_1050-1150MHz.h5
INFO:fpipe.timestream.bandpass_cal:Load bandpass from /home/ycli/data/fanalysis/bandpass/20220213/bandpass_HIMGS_1100+2632.h5
INFO:fpipe.timestream.bandpass_cal:Apply bandpass to Feed 01
INFO:fpipe.ti



=                                        =
=        DONE FOR THE PIPELINE!!         =
=           CONGRATULATIONS!!            =
=                                        =


INFO:tlpipe.pipeline.pipeline:Bandpass_Cal reading data from files:
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0001-0001_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0002-0002_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0003-0003_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0004-0004_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0005-0005_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0006-0006_1150-1250MHz.h5
	/home/ycli/data/fanalysis/raw/20220213/HIMGS_1100+2632_arcdrift0007-0007_1150-1250MHz.h5


In [ ]:
for DATA_Key, DATA, DATE in iter_data_key(DATA_Key_list):
    file_temp = '%s/%s_arcdrift%04d-%04d%s.h5'

    for suffix in band_list:
        input_files = [file_temp%(DATE, DATA, i, i, suffix) for i in range(1, 8)]
        plot_wf['pkat1_input_files'] = ['bpcal/%s'%f for f in input_files]
        plot_wf['pkat1_fig_name']  = 'plot/%s_%s_%s%s'%('bpcal', DATA, DATE, suffix)
        #plot_wf()
        %reset -f array